In [1]:
import keras

Using TensorFlow backend.


In [2]:
print('keras: ', keras.__version__)

keras:  2.0.8


In [3]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import numpy as np
import os
import random
import tensorflow as tf

In [4]:
# Get images
image_raw = img_to_array(load_img('dog.jpg'))
image_raw = np.array(image_raw, dtype=float)
#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=False)

In [5]:
X = rgb2lab(1.0/255*image_raw)[:,:,0]
Y = rgb2lab(1.0/255*image_raw)[:,:,1:]
Y /= 128
X = X.reshape(1, 256, 256, 1)
Y = Y.reshape(1, 256, 256, 2)

In [6]:
def conv_stack(data, filters, strides):
    for i in strides:
        output = Conv2D(filters, (3, 3), strides=i, activation='relu', padding='same')(data)
        output = BatchNormalization()(output)
        return output

#Add inception embedding
img_path = 'dog.jpg'
img = image.load_img(img_path, target_size=(299, 299))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
embed = inception.predict(img)

#Encoder
encoder_input = (InputLayer(input_shape=(None, None, 1)))
encoder_output = conv_stack(encoder_input, 64, [2])
encoder_output = conv_stack(encoder_output, 128, [1, 2])
encoder_output = conv_stack(encoder_output, 256, [1, 2])
encoder_output = conv_stack(encoder_output, 512, [1, 1])
encoder_output = conv_stack(encoder_output, 256, [1])
encoder_output = conv_stack(encoder_output, 128, [1])

#Fusion
# y_mid: (None, 256, 28, 28)
fusion_input = InputLayer(input_shape=(32, 32, 1))
fusion_output = RepeatVector(28 * 28)(fusion_input) # shape: (None, 28*28, 256)
fusion_output = Permute((2, 1))(fusion_output) # shape: (None, 256, 28*28)
fusion_output = Reshape(256, 28, 28)(fusion_output) # shape: (None, 256, 28, 28)
y_concat = Merge(layers=[fusion_output, encoder_output], mode='concat', concat_axis=1) # (None, 512, 28, 28)
fusion_output = Conv2D(256, 1, 1, activation='relu')(y_concat) # (None, 256, 28, 28) and Eq. (5)

#Decoder
fusion_output = (InputLayer(input_shape=(fusion_output.shape)))
decoder_output = UpSampling2D((2, 2))(fusion_output)
decoder_output = conv_stack(decoder_output, 64, [1, 1])(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = conv_stack(decoder_output, 32, [1])(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, fusion_input], outputs=decoder_output)

ValueError: Layer conv2d_204 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.engine.topology.InputLayer'>. Full input: [<keras.engine.topology.InputLayer object at 0x139ac8860>]. All inputs to the layer should be tensors.

In [ ]:
# Finish model
model.compile(optimizer='rmsprop', loss='mse')
model.fit(encoder_input=X,
    fusion_input=embed,
    y=Y,
    batch_size=1,
    epochs=1000)

In [ ]:
print(decoder.evaluate(X, Y, batch_size=1))
output = decoder.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((300, 300, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))